In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from flask import Flask, render_template, request, jsonify

# Load your attendance dataset
dataset = pd.read_csv('attendence_sheet.csv')

# Preprocess the data
dataset['date'] = pd.to_datetime(dataset['date'])
dataset.set_index('date', inplace=True)

# Feature scaling
scaler = MinMaxScaler()
dataset_scaled = scaler.fit_transform(dataset[['attendance']])

# Create sequences for time-series prediction
sequence_length = 10  # You can adjust this based on your data
X, y = [], []
for i in range(len(dataset_scaled) - sequence_length):
    X.append(dataset_scaled[i:i+sequence_length])
    y.append(dataset_scaled[i+sequence_length])

X, y = np.array(X), np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build an LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Flask Web Application
app = Flask(__name__)

# Define routes
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Get input data from the form
        admission_no = request.form['admission_no']
        date = pd.to_datetime(request.form['date'])

        # Prepare input for prediction
        input_data = dataset.loc[dataset['admission_no'] == admission_no]['attendance'].values[-sequence_length:]
        input_data = scaler.transform(input_data.reshape(-1, 1)).reshape(1, sequence_length, 1)

        # Make a prediction
        prediction = model.predict(input_data)
        predicted_attendance = scaler.inverse_transform(prediction)[0][0]

        # You can now use the predicted_attendance in your application

        return render_template('result.html', admission_no=admission_no, date=date, predicted_attendance=predicted_attendance)

if __name__ == '__main__':
    app.run(debug=True)
